<a href="https://colab.research.google.com/github/sim-so/weekly-presentation-season1/blob/main/05_sentence_pair_classification/SNLI_baseline_roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 패키지 다운로드
- torch cuda (GPU 사용하는 경우)
- transformers, datasets

In [ ]:
!pip3 install torch --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116


In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.3 MB/s 
     |████████████████████████████████| 182 kB 63.0 MB/s 
     |████████████████████████████████| 7.6 MB 60.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 5.0 MB/s 
     |████████████████████████████████| 212 kB 66.1 MB/s 
     |████████████████████████████████| 132 kB 72.9 MB/s 
     |████████████████████████████████| 127 kB 78.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


#데이터셋 다운로드
- snli 데이터셋 다운로드

In [ ]:
from datasets import load_dataset
from datasets import Dataset

dataset = load_dataset("snli")

Downloading:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/550152 [00:00<?, ? examples/s]

Dataset snli downloaded and prepared to /root/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# 데이터셋 확인
dataset

DatasetDict({
    test: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 550152
    })
})

# 데이터셋 전처리

## 데이터셋 분리

In [ ]:
train_dataset = dataset['train']
valid_dataset = dataset['validation']
test_dataset = dataset['test']

## 데이터에 이상이 있는지 확인
- label이 -1인 데이터 : 레이블이 주어지지 않은 경우. 삭제하여 진행

In [ ]:
train_dataset.filter(lambda example: example["label"] == -1)

  0%|          | 0/551 [00:00<?, ?ba/s]

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 785
})

In [ ]:
valid_dataset.filter(lambda example: example["label"] == -1)

  0%|          | 0/10 [00:00<?, ?ba/s]

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 158
})

In [ ]:
test_dataset.filter(lambda example: example["label"] == -1)

  0%|          | 0/10 [00:00<?, ?ba/s]

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 176
})

## 토크나이징
1. roberta-base 토크나이저에는 token_type_ids가 없다.
- This is enough for some models to understand where one sequence ends and where another begins. However, other models, such as BERT, also deploy token type IDs (also called segment IDs). They are represented as a binary mask identifying the two types of sequence in the model. [link](https://huggingface.co/docs/transformers/glossary#token-type-ids)
- token_type_ids 없이도 문장 구분을 잘 한다고 하는데 진짜일까...? token_type_ids 유무가 성능에 영향을 미치는지 궁금하다면 bert-base나 electra-base 등 활용해서 실험해보자.
2. transformers의 Dataset이 가지고 있는 method 활용하여 토크나이징 및 전처리
- The fastest way to tokenize your entire dataset is to use the map() function. This function speeds up tokenization by applying the tokenizer to batches of examples instead of individual examples. Set the batched parameter to True: [link](https://huggingface.co/docs/datasets/use_dataset#tokenize-text)
- map() 함수 전반적인 설명 : [link](https://huggingface.co/docs/datasets/process#map)
- 속도가 빠른가 싶어서 써봤는데 pandas로 다 바꿔서 진행하는 게 편했을 것 같다...

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

- padding은 collator 단계로 미루기 (dynamic padding을 할 수 있기 때문)
- tokenizer()에 'return_input_type_ids=True'를 추가할 수도 있지만, 어차피 id가 구분되지 않아서 생략(roberta-base의 경우)

In [ ]:
def tokenization(example):
    return tokenizer(example["premise"], example["hypothesis"], truncation=True)

def encode(dataset):
    dataset = dataset.filter(lambda x: x["label"] != -1)        # 레이블이 없는 데이터 지우기
    dataset = dataset.map(tokenization, batched=True)           # batched=True로 하면 더 빠르다고 함
    dataset = dataset.remove_columns(["premise", "hypothesis"]) # remove 하지 않으면 그대로 남아 있음
    dataset = dataset.rename_column("label", "labels")          # labels로 바꾸지 않고 train 진행했더니 오류가 발생했음
    dataset = dataset.with_format("torch")                      # torch 기준으로 진행
    return dataset

train_encoded = encode(train_dataset)
valid_encoded = encode(valid_dataset)
test_encoded = encode(test_dataset)

  0%|          | 0/551 [00:00<?, ?ba/s]

  0%|          | 0/550 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [ ]:
# encoded 예시
for k, v in train_encoded[0].items():
    print(f"{k} : {v}")

labels : 1
input_ids : tensor([    0,   250,   621,    15,    10,  5253, 13855,    81,    10,  3187,
          159, 16847,     4,     2,     2,   250,   621,    16,  1058,    39,
         5253,    13,    10,  1465,     4,     2])
attention_mask : tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1])


### (optional) 테스트용 샘플 데이터 만들기
- 빠르게 코드를 확인하기 위해 10000개짜리 학습 샘플을 만든다.

In [ ]:
train_encoded_mini = train_encoded.select(range(10000))
train_encoded_mini

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 10000
})

# Data Collator
- huggingface의 DataCollatorWithPadding()을 써도 되고 직접 만들어도 된다.
- 여기서는 batch input sequence의 최대 길이에 맞춰 padding 한다. (= dynamic padding)
- 앞에서 encoding 할 때 한 번에 모델에 들어가는 최대 길이로 padding해도 특별히 상관없다. 

In [ ]:
# custom collator
import numpy as np

class DataCollator():
    def __init__(self, tokenizer, max_length):
        self.max_length = max_length            # 모델에 넣을 수 있는 최대 sequence 길이
        self.tokenizer = tokenizer              # 앞서 encoding에 사용한 토크나이저

    def __call__(self, examples):
        max_input_length = np.max([len(example["attention_mask"]) for example in examples]) # batch input sequences의 최대 길이 확인
        if max_input_length > self.max_length:                                              
            self.padding_length = self.max_length
        else: self.padding_length = max_input_length                                        # 가장 긴 batch input sequence의 길이에 맞춰 padding
        # input sequence가 max_length보다 길면 잘라야 하는데 그 부분은 일단 미루었음 (snli dataset의 경우 encoded 기준 최대 길이 126이라 진행 가능)
        padded_examples = self.tokenizer.pad(examples, max_length=self.padding_length, padding='max_length', return_tensors="pt") 
        return padded_examples

In [ ]:
data_collator_a = DataCollator(tokenizer, 128)

In [ ]:
# huggingface collator
# 학습에 사용한 건 이쪽

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Dataloader (생략)
- 사실 huggingface trainer는 dataloader를 필요로 하지 않는다. (책에서는 lighting 때문에 쓴 듯)
- DataCollator가 잘 돌아가는지만 확인하고 실제 학습에는 사용 안 함

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_dataloader = DataLoader(
    train_encoded, batch_size=16, shuffle=True, collate_fn=data_collator
)

In [ ]:
for step, batch in enumerate(train_dataloader):
    print(batch["input_ids"].shape)
    if step > 5:
        break
# batch 단위로 length(size[1])가 다른 모습 확인

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


torch.Size([16, 55])
torch.Size([16, 39])
torch.Size([16, 52])
torch.Size([16, 48])
torch.Size([16, 38])
torch.Size([16, 47])
torch.Size([16, 36])


#모델 가져오기

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=3, force_download=True) # num_labels=3 하지 않으면 이진분류인 줄 알고 에러 남

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

 Training arguments
 - batch_size=8, pf16=False, 그 외 기본값으로 적용했을 때 예상 학습시간 9시간 이상
 - 이것저것 설정해서 3시간으로 줄임

In [ ]:
batch_size = 16             # default 8
n_epoch = 5                 # 
n_train_dataset = len(train_encoded)
total_iter = int((n_train_dataset * n_epoch) / batch_size)
print(total_iter)

171677


In [ ]:
from transformers import TrainingArguments

# 쓸 수 있는 게 많지만 테스트 중이라 간단히
training_args = TrainingArguments(
    output_dir=f".checkpoints",
    num_train_epochs=n_epoch,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_steps=total_iter // 20,
    logging_steps=total_iter // 20,
    warmup_ratio=0.1,
    evaluation_strategy='steps',
    save_strategy='steps',
    fp16=True,
    load_best_model_at_end=True,
    group_by_length=True        # dynamic padding 사용할 때는 꼭 추가 (비슷한 length의 input sequence끼리 batch를 만듦)
)

In [ ]:
import torch
import numpy as np

# 이것도 일단 적당히 했다가 수정함 (training_args에 evaluation_strategy가 설정되지 않아 evaluation이 되지 않았음)
# 입력받는 evaluation output이 tuple 형태(logits, label_ids)임
def accuracy(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1) # 3개 중에 가장 큰 값의 인덱스 = 예측 클래스
    correct = (preds == labels).sum()
    total = preds.shape[0]
    results = {"accuracy" : correct / total}
    return results

In [ ]:
from transformers import set_seed

set_seed(27)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,       
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_encoded,
    eval_dataset=valid_encoded,
    tokenizer=tokenizer,
    compute_metrics=accuracy
)

Using cuda_amp half precision backend


In [ ]:
trainer.train() # batch size의 default는 8

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 549367
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 171680
  Number of trainable parameters = 124647939
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
8583,0.517200,0.367763,0.865678
17166,0.446100,0.368715,0.865170
25749,0.458400,0.435770,0.862731
34332,0.442300,0.399975,0.864052
42915,0.417300,0.377012,0.879293
51498,0.410500,0.355576,0.876143
60081,0.397800,0.333867,0.881630
68664,0.384000,0.322542,0.888844
77247,0.335300,0.370765,0.884068
85830,0.334500,0.353366,0.892400


***** Running Evaluation *****
  Num examples = 9842
  Batch size = 16
Saving model checkpoint to .checkpoints/checkpoint-8583
Configuration saved in .checkpoints/checkpoint-8583/config.json
Model weights saved in .checkpoints/checkpoint-8583/pytorch_model.bin
tokenizer config file saved in .checkpoints/checkpoint-8583/tokenizer_config.json
Special tokens file saved in .checkpoints/checkpoint-8583/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 9842
  Batch size = 16
Saving model checkpoint to .checkpoints/checkpoint-17166
Configuration saved in .checkpoints/checkpoint-17166/config.json
Model weights saved in .checkpoints/checkpoint-17166/pytorch_model.bin
tokenizer config file saved in .checkpoints/checkpoint-17166/tokenizer_config.json
Special tokens file saved in .checkpoints/checkpoint-17166/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 9842
  Batch size = 16
Saving model checkpoint to .checkpoints/checkpoint-25749
Configuration sav

TrainOutput(global_step=171680, training_loss=0.33880103429343694, metrics={'train_runtime': 17881.3692, 'train_samples_per_second': 153.614, 'train_steps_per_second': 9.601, 'total_flos': 3.946482552891514e+16, 'train_loss': 0.33880103429343694, 'epoch': 5.0})

In [ ]:
output_dir = '/content/snli'
trainer.save_model(output_dir)

Saving model checkpoint to /content/snli
Configuration saved in /content/snli/config.json
Model weights saved in /content/snli/pytorch_model.bin
tokenizer config file saved in /content/snli/tokenizer_config.json
Special tokens file saved in /content/snli/special_tokens_map.json


In [ ]:
!zip -r /content/snli_best.zip /content/snli

  adding: content/snli/ (stored 0%)
  adding: content/snli/tokenizer.json (deflated 72%)
  adding: content/snli/tokenizer_config.json (deflated 48%)
  adding: content/snli/vocab.json (deflated 59%)
  adding: content/snli/pytorch_model.bin (deflated 11%)
  adding: content/snli/merges.txt (deflated 53%)
  adding: content/snli/special_tokens_map.json (deflated 52%)
  adding: content/snli/training_args.bin (deflated 48%)
  adding: content/snli/config.json (deflated 52%)


In [ ]:
from google.colab import files

files.download('/content/snli_best.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Inference
- 이 부분까지는 못 했습니다!

In [ ]:
device = torch.device('cuda:0')

In [ ]:
# 인퍼런스 함수를 만든다
# 여러 개 문장이 들어갈 때 (1) 반복하기 (2) 결과물 저장/기록하기 추가합시다
def inference_fn(premise, hypothesis):
    inputs = tokenizer(
        [(premise, hypothesis)],    #여기만 달라짐
        max_length=128,             #숫자 직접 넣어주자 
        padding="max_length",
        truncation=True,
    )
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v).to(device) for k, v in inputs.items()})
        prob = outputs.logits.softmax(dim=1).to(device) #[0.1, 0.8, 0.1]
  ## 여기도 달라짐 
        entailment_prob = round(prob[0][0].item(), 2)
        neutral_prob = round(prob[0][1].item(), 2)
        contradiction_prob = round(prob[0][2].item(), 2)
        if torch.argmax(prob) == 0: 
            pred = "참 (entailment)"
        elif torch.argmax(prob) == 1:
            pred = "중립 (neutral)"
        else:
            pred = "거짓 (contradiction)"
    return {
        'premise': premise,
        'hypothesis': hypothesis,
        'prediction': pred,
        'entailment_data': f"참 {entailment_prob}",
        'neutral_data': f"중립 {neutral_prob}",
        'contradiction_data': f"거짓 {contradiction_prob}",
        'entailment_width': f"{entailment_prob * 100}%",
        'neutral_width': f"{neutral_prob * 100}%",
        'contradiction_width': f"{contradiction_prob * 100}%",
    }

In [ ]:
# 학습이 덜 되어서 그런가 분류를 잘 못함
inference_fn(test_dataset['premise'][0], test_dataset['hypothesis'][0])

{'premise': 'This church choir sings to the masses as they sing joyous songs from the book at a church.',
 'hypothesis': 'The church has cracks in the ceiling.',
 'prediction': '거짓 (contradiction)',
 'entailment_data': '참 0.01',
 'neutral_data': '중립 0.09',
 'contradiction_data': '거짓 0.9',
 'entailment_width': '1.0%',
 'neutral_width': '9.0%',
 'contradiction_width': '90.0%'}

In [ ]:
inference_fn(test_dataset['premise'][1], test_dataset['hypothesis'][1])

{'premise': 'This church choir sings to the masses as they sing joyous songs from the book at a church.',
 'hypothesis': 'The church is filled with song.',
 'prediction': '참 (entailment)',
 'entailment_data': '참 0.9',
 'neutral_data': '중립 0.07',
 'contradiction_data': '거짓 0.03',
 'entailment_width': '90.0%',
 'neutral_width': '7.000000000000001%',
 'contradiction_width': '3.0%'}

In [ ]:
inference_fn(test_dataset['premise'][2], test_dataset['hypothesis'][2])

{'premise': 'This church choir sings to the masses as they sing joyous songs from the book at a church.',
 'hypothesis': 'A choir singing at a baseball game.',
 'prediction': '거짓 (contradiction)',
 'entailment_data': '참 0.01',
 'neutral_data': '중립 0.05',
 'contradiction_data': '거짓 0.95',
 'entailment_width': '1.0%',
 'neutral_width': '5.0%',
 'contradiction_width': '95.0%'}

In [ ]:
inference_fn(test_dataset['premise'][3], test_dataset['hypothesis'][3])

{'premise': 'A woman with a green headscarf, blue shirt and a very big grin.',
 'hypothesis': 'The woman is young.',
 'prediction': '중립 (neutral)',
 'entailment_data': '참 0.02',
 'neutral_data': '중립 0.97',
 'contradiction_data': '거짓 0.01',
 'entailment_width': '2.0%',
 'neutral_width': '97.0%',
 'contradiction_width': '1.0%'}

In [ ]:
inference_fn(test_dataset['premise'][4], test_dataset['hypothesis'][4])

{'premise': 'A woman with a green headscarf, blue shirt and a very big grin.',
 'hypothesis': 'The woman is very happy.',
 'prediction': '참 (entailment)',
 'entailment_data': '참 0.51',
 'neutral_data': '중립 0.48',
 'contradiction_data': '거짓 0.01',
 'entailment_width': '51.0%',
 'neutral_width': '48.0%',
 'contradiction_width': '1.0%'}

In [ ]:
inference_fn(test_dataset['premise'][10], test_dataset['hypothesis'][10])

{'premise': 'A statue at a museum that no seems to be looking at.',
 'hypothesis': 'There is a statue that not many people seem to be interested in.',
 'prediction': '참 (entailment)',
 'entailment_data': '참 0.53',
 'neutral_data': '중립 0.45',
 'contradiction_data': '거짓 0.02',
 'entailment_width': '53.0%',
 'neutral_width': '45.0%',
 'contradiction_width': '2.0%'}